In [13]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = None

from sklearn import preprocessing
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, f1_score, roc_auc_score
from sklearn.feature_selection import SequentialFeatureSelector as SFS, SelectFromModel
from xgboost import XGBClassifier as XGBC
from xgboost import XGBRegressor as XGBR

In [14]:
merged=pd.read_csv('Mmerged.csv')
mergedT=pd.read_csv('MmergedTest.csv')
mergedH=pd.read_csv('MmergedHoldout.csv')

In [15]:
reg_merged=merged.drop(['id','hosp_admittime','hosp_dischtime','icu_intime','icu_outtime','icu_outcome'], axis=1)
reg_mergedT=mergedT.drop(['id','hosp_admittime','hosp_dischtime','icu_intime','icu_outtime','icu_outcome'],axis=1)
reg_mergedH=mergedH.drop(['id','hosp_admittime','hosp_dischtime','icu_intime','icu_outtime','icu_outcome'],axis=1)

In [16]:
#split X, y

regX_train=reg_merged.drop(['los_icu'], axis=1)
regy_train=reg_merged['los_icu']

regX_test=reg_mergedT.drop(['los_icu'], axis=1)
regy_test=reg_mergedT['los_icu']

regX_holdout=reg_mergedH.drop(['los_icu'], axis=1)
regy_holdout=reg_mergedH['los_icu']

In [17]:
#normalise
num_cols = regX_train.columns[regX_train.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
scaler = preprocessing.StandardScaler()
regX_train[num_cols] = scaler.fit_transform(regX_train[num_cols])

num_cols = regX_test.columns[regX_test.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
scaler = preprocessing.StandardScaler()
regX_test[num_cols] = scaler.fit_transform(regX_test[num_cols])

num_cols = regX_holdout.columns[regX_holdout.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
scaler = preprocessing.StandardScaler()
regX_holdout[num_cols] = scaler.fit_transform(regX_holdout[num_cols])

In [18]:

xgbr = XGBR()

parameters = {
    'n_estimators':np.arange(start=2,stop=20,step=2),
    'max_depth':np.arange(start=2,stop=6,step=1),
    'learning_rate':np.arange(start=0.05,stop=0.4,step=0.05)
}
kfoldCV = KFold(n_splits=5)
Best_xgbr = GridSearchCV(
    xgbr,
    param_grid=parameters,
    cv=kfoldCV,
)

Best_xgbr.fit(regX_train,regy_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    i...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None, ...),
             param_grid={'learning_rate': array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35]),
                         'max_depth': array([2, 3, 4, 5]),
                         'n_estimators': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18])})

In [19]:
Best_xgbr.best_estimator_

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.3, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=18, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [20]:
xgbr_pred = Best_xgbr.predict(regX_test)
xgbr_MSE = mean_squared_error(regy_test, xgbr_pred)

xgbr_MSE

32.30179982595055

In [21]:
xgbr_predH = Best_xgbr.predict(regX_holdout)
xgbr_MSE_H = mean_squared_error(regy_holdout, xgbr_predH)

xgbr_MSE_H

27.57051605695562